# Language Classification

We will learn to classify phrases as Spanish or English.

First, lets get all the data.

In [1]:
import requests, re

spanish_novels = ['http://www.gutenberg.org/cache/epub/2000/pg2000.txt']
english_novels = ['https://www.gutenberg.org/files/46787/46787-0.txt']

spanish_sentences = [s for novel in spanish_novels 
                     for s in re.sub(r'\s+', ' ', requests.get(novel).text).split('.')[20:-20]]
english_sentences = [s for novel in english_novels
                     for s in re.sub(r'\s+', ' ', requests.get(novel).text).split('.')[20:-20]]

spanish_sentences[:2], english_sentences[:2]

([' AL DUQUE DE BÉJAR, marqués de Gibraleón, conde de Benalcázar y Bañares, vizconde de La Puebla de Alcocer, señor de las villas de Capilla, Curiel y Burguillos En fe del buen acogimiento y honra que hace Vuestra Excelencia a toda suerte de libros, como príncipe tan inclinado a favorecer las buenas artes, mayormente las que por su nobleza no se abaten al servicio y granjerías del vulgo, he determinado de sacar a luz al Ingenioso hidalgo don Quijote de la Mancha, al abrigo del clarísimo nombre de Vuestra Excelencia, a quien, con el acatamiento que debo a tanta grandeza, suplico le reciba agradablemente en su protección, para que a su sombra, aunque desnudo de aquel precioso ornamento de elegancia y erudición de que suelen andar vestidas las obras que se componen en las casas de los hombres que saben, ose parecer seguramente en el juicio de algunos que, continiéndose en los límites de su ignorancia, suelen condenar con más rigor y menos justicia los trabajos ajenos; que, poniendo los oj

In [ ]:
from __future__ import print_function
import mxnet as mx
import numpy as np
from mxnet import nd, autograd, sym
from mxnet import gluon

ctx = mx.cpu()

data = np.array([(s, 1) for s in spanish_sentences] + [(s, 0) for s in english_sentences])

layer = mx.gluon.rnn.LSTM(100, 3)
net = mx.gluon.nn.Dense(1)
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

layer.initialize(ctx=ctx)
net.collect_params().initialize(ctx=ctx)

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})
for epoch in range(10):
    np.random.shuffle(data)
    losses = []
    for s, l in data:
        if len(s) == 0:
            continue
        
        x = nd.array([ord(c) for c in s]).reshape((-1, 1, 1))
        with autograd.record():
            output = layer(x)
            pred = net(output[output.shape[0]-1, :, :])
            loss = softmax_cross_entropy(pred, nd.array([l]))
        losses.append(loss.asscalar())
        trainer.step(1, ignore_stale_grad=True)
    print("Loss:", np.mean(losses), "+-", np.std(losses))